In [ ]:
!pip install -q lime
!pip install -q matplotlib_venn_wordcloud

!pip install -q transformers
!pip install -q sentence-transformers

from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Tesi\ Magistrale\ 204963/Progetto

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Tesi Magistrale 204963/Progetto


In [ ]:
import numpy as np
import pickle
import nltk
import random
import pandas as pd
import os
import itertools
import torch
import math
import re
import scipy.sparse as sparse
from sklearn.feature_extraction.text import CountVectorizer

from text_interpretable import LemmaTokenizer
from coherence import compute_score
from autoencoders.model import VAE, DAE, AAE, reparameterize
from autoencoders.vocab import Vocab
from autoencoders.noise import noisy
from autoencoders.utils import set_seed, strip_eos
from autoencoders.batchify import get_batches
from classifier.classifier_nn import build_nn

from sklearn.model_selection import train_test_split

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

#from sentence_transformers import SentenceTransformer, util
#SentenceTransformer('stsb-roberta-large')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def load_sent(path):
    sents = []
    with open(path) as f:
        for line in f:
          sents.append(line)
    return sents

def get_model(path):
    ckpt = torch.load(path)
    train_args = ckpt['args']
    model = {'dae': DAE, 'vae': VAE, 'aae': AAE}[train_args.model_type](
        vocab, train_args).to(device)
    model.load_state_dict(ckpt['model'])
    model.flatten()
    model.eval()
    return model

In [ ]:
dataset = 'yelp'

In [ ]:
if dataset == 'yahoo':
  sents = pickle.load(open("data/yahoo_answers_csv/test_2_processed.pickle",'rb'))
  y = pickle.load(open("data/yahoo_answers_csv/test_2_processed_y.pickle",'rb'))

  path = 'checkpoints/yahoo/daae'

  cv_d = pickle.load(open("tests/yahoo/cv_d.pickle",'rb'))
  score_slime = pickle.load(open('tests/yahoo/score_fabrizio_correzione.pickle', 'rb'))

  sents_vocab = sents.copy()

if dataset == 'yelp':
  sents = load_sent(os.path.join('data/yelp/sentiment/1000.pos'))
  n_pos = len(sents)
  sents.extend(load_sent(os.path.join('data/yelp/sentiment/1000.neg')))
  y = np.ones(len(sents), dtype=int)
  y[n_pos:] = 0

  sents_vocab = np.append(sents,load_sent(os.path.join('data/yelp/train.txt')),axis=0)

  path = 'checkpoints/yelp/daae'

  cv_d = pickle.load(open("tests/tree/model_2/cv_d.pickle",'rb'))
  score_slime = pickle.load(open('tests/tree/model_2/score_fabrizio_distance_2.pickle', 'rb'))

# ALTRO DATASET #
if dataset == 'twitter':
  sents = load_sent( os.path.join('data/twitter/twitter_pos.txt') )
  n_pos = len(sents)
  sents.extend(load_sent(os.path.join('data/twitter/twitter_neg.txt')))
  y = np.ones(len(sents), dtype=int)
  y[n_pos:] = 0

  sents_vocab = load_sent(os.path.join('data/twitter/twitter_train.txt'))
  

print("sents_vocab =", len(sents_vocab))
print("sents len =",len(sents))
print("y len =",len(y))

sents_vocab = 202000
sents len = 2000
y len = 2000


Lemmatizer e rimozione stopwords

In [ ]:
stopwords = pickle.load(open("stop_words_yelp.pickle",'rb'))

lt = LemmaTokenizer(split_expression=r'\W+', stop_words=stopwords)

data_lt = lt(sents_vocab)

In [ ]:
for i in range(4):
    n = random.randint(a=0, b=len(data_lt))
    print(n,')',sents_vocab[n][:-1],'===>', data_lt[n])

2088 ) 'before long , the film starts playing like general hospital crossed with a saturday night live spoof of dog day afternoon . ===> long film start play like general hospital cross saturday night live spoof dog day afternoon
7171 ) "this movie may not have the highest production values you've ever seen , but it's the work of an artist , one whose view of america , history and the awkwardness of human life is generous and deep . ===> movie may not have high production value ve ever see but it work an artist one whose view america history awkwardness human life generous deep
1345 ) '[stevens is] so stoked to make an important film about human infidelity and happenstance that he tosses a kitchen sink onto a story already overladen with plot conceits . ===> stevens stoke make an important film human infidelity happenstance he toss kitchen sink onto story already overladen plot conceit
5658 ) 'as if to prove a female director can make a movie with no soft edges , kathryn bigelow offers

CountVectorizer Single Word
===

In [ ]:
my_cv = CountVectorizer(binary=True)
my_cv.fit(data_lt)

#data_cv = my_cv.transform(lt(sents)) 
data_cv = my_cv.transform(data_lt)

print("my vocab size =>",len(my_cv.vocabulary_))
print("vocab size slime =>",len(cv_d.vocabulary_))

print(data_cv.shape)

In [ ]:
# DIFFERENZE TRA VOCAB MIO E VOCAB SLIME
diff = (set(my_cv.vocabulary_))-set(cv_d.vocabulary_)
print('num differences :',len(diff))

Batch splitting and score computing

In [ ]:
class_folder = 'classifier/'+dataset+'/'

if dataset == 'yahoo':
  mnb = pickle.load(open(os.path.join(class_folder, 'mnb.pickle'), 'rb'))
  vec = pickle.load(open(os.path.join(class_folder, 'vec.pickle'), 'rb'))
  classifier_fn = lambda s: mnb.predict_proba(vec.transform(s))

if dataset == 'yelp':
  vocab = Vocab(os.path.join(path, 'vocab.txt'))

  cuda = torch.cuda.is_available()
  device = torch.device("cuda" if cuda else "cpu")
  model = get_model(os.path.join(path, 'model.pt'))

  vec = pickle.load(open(os.path.join(class_folder, 'vec_nn.pickle'), 'rb'))
  classifier = build_nn(len(vec.vocabulary_))
  classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'))
  #classifier_fn = lambda s: classifier.predict(vec.transform(s).toarray())

  def classifier_fn(s):
    p = classifier.predict(vec.transform(s).toarray())
    return np.append(1-p, p, axis=1) # prob di essere neg, prob di essere pos

print(dataset)
pre_classes = classifier_fn(sents_vocab)
pre_classes = np.where(pre_classes[:,0]>=0.5, 0, 1)

In [ ]:
from math import ceil

batch_dim = 200

sum_pos = np.zeros(len(cv_d.get_feature_names()))
sum_pos = sum_pos.reshape((1, sum_pos.shape[0]))
sum_tot = np.zeros(len(cv_d.get_feature_names()))
sum_tot = sum_tot.reshape((1, sum_tot.shape[0]))

for i in range(ceil(data_cv.shape[0]/batch_dim)):
  sum_tot += np.sum( data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )
  sum_pos += np.sum( pre_classes[i*batch_dim : (i+1)*batch_dim] * data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )

sum_pos = np.squeeze(sum_pos)
sum_tot = np.squeeze(sum_tot)

print(sum_tot.shape)
print(sum_pos.shape)

In [ ]:
score = compute_score(sum_pos=sum_pos, sum_tot=sum_tot)
print("my score",score.shape)
print("slime score",score_slime.shape)
print()
print('my score sum',np.sum(score))
print('slime score sum',np.sum(score_slime))

In [ ]:
df = pd.DataFrame([score, score_slime], index=['My_score', 'SLIME_score'] )
df.columns = cv_d.get_feature_names()
df

In [ ]:
with open("mytest/"+dataset+"/vocab_score",'wb') as f:
  pickle.dump(score, f)
with open("mytest/"+dataset+"/my_CV",'wb') as f:
  pickle.dump(my_cv, f)

YELP : CountVectorizer N-Gram
===

Definisci modello e calcola occorrenze
---

In [ ]:
new_CV = CountVectorizer(ngram_range=(1,3), binary=True)
data_cv = new_CV.fit_transform(data_lt)
len(new_CV.vocabulary_)

#pickle.dump( new_CV, open('mytest/'+dataset+'/new_CV.pickle','wb') )

821328

In [ ]:
class_folder = 'classifier/'+dataset+'/'

if dataset == 'yahoo':
  mnb = pickle.load(open(os.path.join(class_folder, 'mnb.pickle'), 'rb'))
  vec = pickle.load(open(os.path.join(class_folder, 'vec.pickle'), 'rb'))
  classifier_fn = lambda s: mnb.predict_proba(vec.transform(s))

if dataset == 'yelp':
  vocab = Vocab(os.path.join(path, 'vocab.txt'))

  cuda = torch.cuda.is_available()
  device = torch.device("cuda" if cuda else "cpu")
  model = get_model(os.path.join(path, 'model.pt'))

  vec = pickle.load(open(os.path.join(class_folder, 'vec_nn.pickle'), 'rb'))
  classifier = build_nn(len(vec.vocabulary_))
  classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'))
  #classifier_fn = lambda s: classifier.predict(vec.transform(s).toarray())

  def classifier_fn(s):
    p = classifier.predict(vec.transform(s).toarray())
    return np.append(1-p, p, axis=1) # prob di essere neg, prob di essere pos

print(dataset)
pre_classes = classifier_fn(sents_vocab)
pre_classes = np.where(pre_classes[:,0]>=0.5, 0, 1)

yelp


In [ ]:
from math import ceil

batch_dim = 1000

gow_sum_pos = np.zeros(len(new_CV.get_feature_names()))
gow_sum_pos = gow_sum_pos.reshape((1, gow_sum_pos.shape[0]))
gow_sum_tot = np.zeros(len(new_CV.get_feature_names()))
gow_sum_tot = gow_sum_tot.reshape((1, gow_sum_tot.shape[0]))

for i in range(ceil(data_cv.shape[0]/batch_dim)):
  gow_sum_tot += np.sum( data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )
  gow_sum_pos += np.sum( pre_classes[i*batch_dim : (i+1)*batch_dim] * data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )

gow_sum_pos = np.squeeze(gow_sum_pos)
gow_sum_tot = np.squeeze(gow_sum_tot)

print(gow_sum_tot.shape)
print(gow_sum_pos.shape)

Calcolo score
---

In [ ]:
gow_score = compute_score(sum_pos=gow_sum_pos, sum_tot=gow_sum_tot)
with open('mytest/'+dataset+'/GOW_score.pickle','wb') as f:
  pickle.dump(gow_score,f)
  
df = pd.DataFrame([gow_score], index=['My_score'] )
df.columns = new_CV.get_feature_names()
df_copy = df.copy()
df_copy = df_copy.max().sort_values(ascending=False)
df_copy[:50]

YAHOO: CountVectorized N-gram
====

Definisci modello e calcola occorrenze
---

In [ ]:
new_CV = CountVectorizer(ngram_range=(1,3), binary=True)
data_cv = new_CV.fit_transform(data_lt)
len(new_CV.vocabulary_)

#pickle.dump( new_CV, open('mytest/'+dataset+'/new_CV.pickle','wb') )

In [ ]:
class_folder = 'classifier/'+dataset+'/'

if dataset == 'yahoo':
  mnb = pickle.load(open(os.path.join(class_folder, 'mnb.pickle'), 'rb'))
  vec = pickle.load(open(os.path.join(class_folder, 'vec.pickle'), 'rb'))
  classifier_fn = lambda s: mnb.predict_proba(vec.transform(s))

if dataset == 'yelp':
  vocab = Vocab(os.path.join(path, 'vocab.txt'))

  cuda = torch.cuda.is_available()
  device = torch.device("cuda" if cuda else "cpu")
  model = get_model(os.path.join(path, 'model.pt'))

  vec = pickle.load(open(os.path.join(class_folder, 'vec_nn.pickle'), 'rb'))
  classifier = build_nn(len(vec.vocabulary_))
  classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'))
  #classifier_fn = lambda s: classifier.predict(vec.transform(s).toarray())

  def classifier_fn(s):
    p = classifier.predict(vec.transform(s).toarray())
    return np.append(1-p, p, axis=1) # prob di essere neg, prob di essere pos

print(dataset)
pre_classes = classifier_fn(sents_vocab)
pre_classes = np.where(pre_classes[:,0]>=0.5, 0, 1)

In [ ]:
from math import ceil

batch_dim = 1000

gow_sum_pos = np.zeros(len(new_CV.get_feature_names()))
gow_sum_pos = gow_sum_pos.reshape((1, gow_sum_pos.shape[0]))
gow_sum_tot = np.zeros(len(new_CV.get_feature_names()))
gow_sum_tot = gow_sum_tot.reshape((1, gow_sum_tot.shape[0]))

for i in range(ceil(data_cv.shape[0]/batch_dim)):
  gow_sum_tot += np.sum( data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )
  gow_sum_pos += np.sum( pre_classes[i*batch_dim : (i+1)*batch_dim] * data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )

gow_sum_pos = np.squeeze(gow_sum_pos)
gow_sum_tot = np.squeeze(gow_sum_tot)

print(gow_sum_tot.shape)
print(gow_sum_pos.shape)

Calcolo score
---

In [ ]:
gow_score = compute_score(sum_pos=gow_sum_pos, sum_tot=gow_sum_tot)
with open('mytest/'+dataset+'/GOW_score.pickle','wb') as f:
  pickle.dump(gow_score,f)
  
df = pd.DataFrame([gow_score], index=['My_score'] )
df.columns = new_CV.get_feature_names()
df_copy = df.copy()
df_copy = df_copy.max().sort_values(ascending=False)
df_copy[:50]

TWITTER: CountVectorized N-gram
===

In [ ]:
new_CV = CountVectorizer(ngram_range=(1,3), binary=True, max_features=820000)
data_cv = new_CV.fit_transform(data_lt)
len(new_CV.vocabulary_)

pickle.dump( new_CV, open('mytest/'+dataset+'/new_CV.pickle','wb') )

In [ ]:
class_folder = 'classifier/'+dataset+'/'

if dataset == 'twitter':
  vec = pickle.load(open(os.path.join(class_folder, 'vec_nn.pickle'), 'rb'))
  classifier = build_nn(len(vec.vocabulary_))
  try:
    classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'))
  except:
    classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'),'rb')
  #classifier_fn = lambda s: classifier.predict(vec.transform(s).toarray())

def classifier_fn(s):
    chunks = [s[x:x+500] for x in range(0, len(s), 500)]
    result = []
    for chunk in chunks:
      p = classifier.predict(vec.transform(chunk).toarray())
      result.extend( np.append(1-p,p, axis=1) )
    del chunk
    del chunks
    return np.asarray(result)

print(dataset)
pre_classes = classifier_fn(sents_vocab)
#pre_classes = np.where(pre_classes[:,0]>=0.5, 0, 1)
pre_classes = np.squeeze(pre_classes)
pre_classes = np.argmax(pre_classes, axis=1)

movie


In [ ]:
from math import ceil

batch_dim = 712

gow_sum_pos = np.zeros(len(new_CV.get_feature_names()))
gow_sum_pos = gow_sum_pos.reshape((1, gow_sum_pos.shape[0]))
gow_sum_tot = np.zeros(len(new_CV.get_feature_names()))
gow_sum_tot = gow_sum_tot.reshape((1, gow_sum_tot.shape[0]))

for i in range(ceil(data_cv.shape[0]/batch_dim)):
  gow_sum_tot += np.sum( data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )
  gow_sum_pos += np.sum( pre_classes[i*batch_dim : (i+1)*batch_dim] * data_cv[i*batch_dim : (i+1)*batch_dim].todense(), axis=0 )

gow_sum_pos = np.squeeze(gow_sum_pos)
gow_sum_tot = np.squeeze(gow_sum_tot)

print(gow_sum_tot.shape)
print(gow_sum_pos.shape)

(209268,)
(209268,)


In [ ]:
gow_score = compute_score(sum_pos=gow_sum_pos, sum_tot=gow_sum_tot)
with open('mytest/'+dataset+'/GOW_score.pickle','wb') as f:
  pickle.dump(gow_score,f)
  
df = pd.DataFrame([gow_score], index=['My_score'] )
df.columns = new_CV.get_feature_names()
df_copy = df.copy()
df_copy = df_copy.max().sort_values(ascending=False)
df_copy[:50]

,_num_,_num_ _num_,_num_ _num_ _num_,_num_ _num_ _num_st,_num_ _num_ an,_num_ _num_ but,_num_ _num_ film,_num_ _num_ hour,_num_ _num_ league,_num_ _num_ million,_num_ _num_ minute,_num_ _num_ new,_num_ _num_ period,_num_ _num_ philosophical,_num_ _num_ powerpuff,_num_ _num_ screen,_num_ _num_ terrorist,_num_ _num_ time,_num_ _num_st,_num_ _num_st century,_num_ _num_ths,_num_ _num_ths it,_num_ actor,_num_ actor appear,_num_ aim,_num_ aim specifically,_num_ alien,_num_ alien plucky,_num_ also,_num_ also happens,_num_ alternative,_num_ alternative housing,_num_ an,_num_ an antique,_num_ an uneasy,_num_ analyze,_num_ analyze promise,_num_ animation,_num_ animation wild,_num_ armenia,...,zone arm,zone arm nothing,zone attempt,zone attempt be,zone bale,zone bale reduce,zone be,zone be laud,zone but,zone but it,zone episode,zone give,zone give voice,zone have,zone have center,zone honest,zone honest enough,zone left,zone left me,zone offer,zone offer win,zone ordinance,zone ordinance protect,zone sympathize,zone sympathize terrorist,zoolander,zoolander thought,zoolander thought rather,zoom,zoom it,zoom it actually,zucker,zucker brother,zucker brother abraham,zwick,zwick heartfelt,zwick heartfelt hilarious,zwick it,zwick it sitcom,zzzzzzzzz
My_score,0.13423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
